In [34]:
import json
import numpy as np
from tqdm.auto import trange, tqdm
import re
import pandas as pd


def tn_to_arb(line):
  fline = ''
  for char in line:
    if ord(char) >= ord('๐') and ord(char) <= ord('๙'):
      fline += str(ord(char) - ord('๐'))
    else:
      fline += char
  return fline

def is_bullet(String):
  regx = [('\d+\.0', 0),
   ('\d+\)', 8),
   ('กิจกรรม', 7),
   ('\(\d*(\.?\d*)*\)', 10),
   ('[1-9](\.[1-9]){5}\D+', 6),
   ('[1-9](\.[1-9]){4}\D+', 5),
   ('[1-9](\.[1-9]){3}\D+', 4),
   ('\d+(\.[1-9]){2}\D+', 3),
   ('[1-9]\.[1-9]', 2),
   ('\d+\.', 1),
   ('\d+![\.|\d+]', 9)]

  for r, l in regx:
    if re.match(r, String):
      return l
  return 0

def get_level_of_line(String):

  regx = [('\d+(\.\d+){2}\)', 10),
   ('\d+\.\d+\)', 9),
   ('\d+\)', 8),
   ('กิจกรรม', 7),
   ('\(\d*(\.?\d*)*\)', 12),   # (x), (x.x), (x.x.x)
   ('\d+(\.\d+){5}\D+', 6),
   ('\d+(\.\d+){4}\D+', 5),
   ('\d+(\.\d+){3}\D+', 4),
   ('\d+(\.\d+){2}\D+', 3),
   ('\d+\.\d+\D+', 2),
   ('\d+\.\D+', 1),
   ('\d+.', 11)]

  for r, l in regx:
    if re.match(r, String):
      return l
  return 0

In [35]:
def to_gregorian(input):
  if input:
    return int(input) - 543
  else:
    return input

In [47]:
class ThaiBudget:
  def __init__(self, json_path):
    with open(json_path) as json_file:
      self._data_ = json.load(json_file)
      self.docs = [ref_doc for ref_doc in self._data_]
  
  def get_doc(self, ref_doc):
    return Doc(self._data_[ref_doc], ref_doc)


class Doc:
  def page(self, num_page, true_num_page=False):
    if true_num_page:
      npage = num_page + self.start
      return Page(npage, self._raw_page[npage], num_page)
    else:
      return Page(num_page, self._raw_page[num_page])

  @property
  def index_page(self):
    id_pages = []
    for i in range(30):
      page = self.page(i).get_text_lines()
      if page != None:
        text = ''.join(page)
        if re.findall('\d+ (ถึง) \d+', text):
          id_pages.append(i)
        elif id_pages:
          break
    return id_pages

  @property
  def ministry(self):
    ministry = []
    lines = self.page(0).get_lines(yPos=True, line_tolerance=23)
    for tline, ypline in lines:
      if (np.array([1250, 1400]) < ypline).any() and (ypline < np.array([2000, 2100])).any():
        ministry.append(' '.join([tn_to_arb(block) for block in tline if block not in '*-']))
    return ministry

  @property
  def range(self):
    page_rage = []
    ministry = ''
    tempdict = {}
    stw = [ministr[:int(len(ministr)*0.6)]for ministr in self.ministry]
    for i in self.index_page:
      lines = self.page(i).get_text_list_lines()
      for line in lines:
        if [sw for sw in stw if ' '.join(line).startswith(sw)]:
          ministry = [word for word in line if word != 'ถึง' and not word.isdigit() and word != 'หน้า']
          ministry = ' '.join(ministry)
        if re.match('\(\d+\)', line[0]):
          if tempdict:
            page_rage.append(tempdict)
            tempdict = {}
          tempdict['ministry'] = ministry
          tempdict['budget_unit'] = ' '.join([word for word in line[1:] if word != 'ถึง' and not word.isdigit()])
        if '7. รายละเอียดงบประมาณจำแนกตามแผนงาน' in ' '.join(line):
          start = line[-1]
        if '8. รายงานสถานะและแผนการใช้จ่ายเงินนอกงบประมาณ' in ' '.join(line):
          tempdict['range'] = (int(start), int(line[-1]))
    return page_rage

  @property
  def start(self):
    idpage = self.index_page
    fstpage = idpage[-1] + 1
    while self.page(fstpage).get_blocks() == None:
      fstpage += 1
    return fstpage - 1
  
  def get_budgetary_unit(self, minis):
    budumi = []
    for budu in self.range:
      if minis == budu['ministry']:
        budumi.append({'budget_unit': budu['budget_unit'],
                      'range': budu['range']})
      
    return budumi

  def get_ministry(self, budge_u):
    for budu in self.range:
      if budge_u == budu['budget_unit']:
        return {'ministry': budu['ministry'],
                      'range': budu['range']}
      else:
        return None

  def __init__(self, raw_pages, ref_doc):
    self.fiscal_year, self.chabab, self.lem = ref_doc.split('.')
    self._raw_page = raw_pages
    self.ref_doc = ref_doc
    self.len = len(raw_pages)

class Page:
  def get_blocks(self):
    if self._page == []:
      return None
    blocks = []
    for block in self._page:
      coord = np.array([[coo['x'], coo['y']] for coo in block['vertices']], dtype=np.uint32)
      crd = self._simplify_coordinates(block['vertices'])
      blocks.append({'text': block['description'],
                    'sim_coord': crd})
    return blocks

  @property
  def xSigPos(self):
    for line in self.get_lines():
      text = [block['text'] for block in line]
      scord = [block['sim_coord'] for block in line]
      for scrd, txt in zip(scord, text):
        if txt == 'บาท' and (scrd[0] > 1340).all() and (scrd[1] > 190).all():
          return scrd[0]

  def _simplify_coordinates(self, coordinate):
    if self._page == []:
      return None
    coord = np.array([[coo['x'], coo['y']] for coo in coordinate], dtype=np.uint32)
    x = [i[0] for i in np.sort(coord,axis=0)]
    y = [i[1] for i in np.sort(coord,axis=0)]
    xy = np.array([x[:2],x[2:], y[:2], y[2:]]).sum(axis=1)//2
    return np.array([xy[:2], xy[2:]], dtype=np.uint32)
  
  def _ypos_mean(self, list_of_blocks):
    if self._page == []:
      return None
    return np.mean([bx['sim_coord'][1] for bx in list_of_blocks], axis=0)

  def get_lines(self, line_tolerance=None, yPos=False):
    line_tolerance = self.dline_tolerance if line_tolerance == None else line_tolerance
    if self._page == []:
      return None
    lines = []
    for b in self.get_blocks():
      if lines == []:
        lines.append([b])
      else:
        cm = b['sim_coord'][1]
        found = 0
        for line in lines:
          diff = abs(cm - self._ypos_mean(line))
          if diff[1] < line_tolerance:
            found = 1
            line.append(b)
            break
        if not found:
          lines.append([b])
    lines = [sorted(line, key = lambda i: i['sim_coord'][0][0]) for line in lines]
    if not yPos:
      return lines
    else:
      return [([block['text'] for block in line], self._ypos_mean(line).tolist()) for line in lines]

  def get_text_lines(self):
    if self._page == []:
      return []
    text_lines = []
    for line in self.get_text_list_lines():
      text_lines.append(' '.join(line))
    return text_lines

  def get_text_list_lines(self):
    if self._page == []:
      return []
    text_lines = []
    for line in self.get_lines():
      text_lines.append([l['text'] for l in line])
    return text_lines    

  def xpos_text_lines(self, line_tolerance=None):
    if self._page == []:
      return None
    xpos_lines = []
    text_lines = []
    lines = self.get_lines() if line_tolerance == None else self.get_lines(line_tolerance)
    for line in lines:
      ax = np.array([bx['sim_coord'] for bx in line])
      xpos_lines.append(ax)
      text_lines.append([bx['text'] for bx in line])
    return  xpos_lines, text_lines

  def __init__(self, index_page, page, pdfpage=None):
    self.dline_tolerance = 23
    self.index_page = index_page
    self.pdfpage = pdfpage
    self._page = page[1:]

In [37]:
def labelling(coord, text):
  labels = []
  xpos = []
  xbltpos = 0
  fscy = [txt for txt in text if txt in ['ตั้งงบประมาณ', 'ผูกพันงบประมาณ', 'วงเงินทั้งสิ้น']]
  if ''.join(text).startswith('เงินนอกงบประมาณ'):
    return None
  if ''.join(text).startswith('(ชดเชยงบประมาณที่พับไป'):
    return None
  if ''.join(text).startswith('รายการบุคลากรภาครัฐ'):
    return None

  for crd, txt in zip(coord, text):
    xpos += crd[0].tolist()
    if 'รายละเอียดงบประมาณจำแนกตามงบรายจ่าย' in text:
      return None

    elif (crd[1] < 130).all():
      labels.append('NPG')
    elif (np.array([len(txt) for txt in text]) < 5).all():
        print('🚨 trash line',text)
        return None
    elif (crd[0] < 375).all() and is_bullet(txt) and not fscy and len(labels) + 1 < len(text) and text[len(labels) + 1] not in ['ล้านบาท', 'บาท', 'รายการ', 'หน่วย']:
      labels.append('BLT')
      xbltpos = crd[0].tolist()[0]
    elif txt == 'ผลผลิต':
      labels.append('OTP')
    elif txt == 'โครงการ':
      labels.append('PRJ')
    elif (crd[0] > 1330).any():
      if re.match(r'[0-9]{1,3}(,[0-9]{1,3})*', txt) and txt.replace(',','').isdigit():
        labels.append('AMT')
      else:
        labels.append('BTH')
    elif (crd[0] > 1000).all() and fscy:
      if re.match(r'[0-9]{1,3}(,[0-9]{1,3})*', txt):
        labels.append('FAMT')
      else:
        labels.append('FBTH')
    elif fscy and txt.isdigit():
        labels.append('FYR')
    else:
        labels.append('NMT')
  return xbltpos if xbltpos else np.min(xpos), text, labels 


In [38]:
def clean_prefix(textlev, labellev):
  otext = []
  olabel = []
  for txt, lbel in zip(textlev, labellev):
    txtlbel = [{'text': text, 'label': label} for text, label in zip(txt, lbel)]
    if 'BLT' in lbel:
      bltindex = lbel.index('BLT')
      txt = txt[bltindex:]
      lbel = lbel[bltindex:]
    if 'FBTH' in lbel :
      if 'ปี' in txt and 'FBTH' in lbel:
        fyindex = txt.index('ปี')
        txt = txt[fyindex:]
        lbel = lbel[fyindex:]
    otext.append(txt)
    olabel.append(lbel)
  return otext, olabel

In [39]:
def get_budget_plan(plist):
  npagelist, poslist, lineslist = [], [], []
  for ipge in plist:
    tmppos, tmptext = ipge.xpos_text_lines()
    poslist += tmppos
    lineslist += tmptext
    npagelist += [ipge.pdfpage] * len(tmptext)

  budget_plan = ''
  for coord, line in zip(poslist, lineslist):
    if [block for block in line if re.match('7\.[1-9]\.[1-9]', block)]:
      return budget_plan
    elif [block for block in line if block.startswith('แผนงาน')]:
      for bcrd, btext in zip(coord, line):
        if (bcrd[0] < 1200).all() and not re.match('7\.[1-9]', btext):
          budget_plan += btext
    elif budget_plan:
      budget_plan += ' '.join(line)

In [40]:
def clean_entry(plist):
  pstart = curr_doc.start
  if len(plist) > 1:
    allxpos = [ipage.xSigPos.tolist()[0] for ipage in plist] 
    xdiffl = np.subtract(allxpos[0], allxpos)
  else:
    xdiffl = [0]
  npagelist, poslist, textlist = [], [], []
  for ipge, xdiff in zip(plist, xdiffl):
      tmppos, tmptext = ipge.xpos_text_lines()
      poslist += [np.add(tmpos, [[xdiff, xdiff], [0, 0]]) for tmpos in tmppos]
      textlist += tmptext
      npagelist += [ipge.pdfpage] * len(tmptext)

  xposlev = []
  textlev = []
  labellev = []
  npagelev = []

  for npage, coord, text in zip(npagelist, poslist, textlist):
    olabel = labelling(coord, text)
    if olabel == None:
      continue
    minxpos, textin, labels = olabel
    if 'NPG' in labels:
      continue
    else:
      xposlev.append(minxpos)
      textlev.append(text)
      labellev.append(labels)
      npagelev.append(npage)

  textlev, labellev = clean_prefix(textlev, labellev)

  oxposlev, otextlev, olabellev, onpagelev = [], [], [], []
  for xpos, text, label, npage in zip(xposlev, textlev, labellev, npagelev):
    if oxposlev:
      if 'BTH' in olabellev[-1] or 'FBTH' in olabellev[-1]:
        if 'FY' in label or 'FBTH' in label:
          xpos = np.int16(9990)
        if 'BTH' in label or 'FBTH' in label or 'BLT' in label:
          oxposlev.append(xpos)
          otextlev.append(text)
          olabellev.append(label)
          onpagelev.append(npage)
        else:
          if textlev.index(text) + 1 < len(labellev):
            if 'BTH' in labellev[textlev.index(text) + 1] and 'BLT' in labellev[textlev.index(text) + 1]:
              otextlev[-1] += text
              olabellev[-1] += label
            elif 'BLT' not in labellev[textlev.index(text) + 1]:
              oxposlev.append(xpos)
              otextlev.append(text)
              olabellev.append(label)
              onpagelev.append(npage)
            else:
              print('🚨',npage, xpos, [(txt, lbel) for txt, lbel in zip(text, label)])
          else:
            otextlev[-1] += text
            olabellev[-1] += label
      elif 'BLT' in olabellev[-1]:
        if 'BLT' not in label:
          otextlev[-1] += text
          olabellev[-1] += label
        else:
          oxposlev.append(xpos)
          otextlev.append(text)
          olabellev.append(label)
          onpagelev.append(npage)
      else:
        otextlev[-1] += text
        olabellev[-1] += label
    else:
      oxposlev.append(xpos)
      otextlev.append(text)
      olabellev.append(label)
      onpagelev.append(npage)

  xposlev, textlev, labellev, npagelev = oxposlev, otextlev, olabellev, onpagelev
  return oxposlev, otextlev, olabellev, onpagelev
  
  return get_entry(xposlev, textlev, labellev, npagelev)

In [46]:
 def get_entry(cleaned_entry):
  xposlev, textlev, labellev, npagelev = cleaned_entry
  iserrorlev = [False] * len(xposlev)
  loglev = [''] * len(xposlev)

  for text, label in zip(textlev, labellev):
    if 'BLT' not in label and 'FBTH' not in label and 'OTP' not in label and 'PRJ' not in label:
      if is_bullet(text[0]):
        label[0] = 'BLT'
      else:
        print('🚨 [1] is not \'BLT\': ', text)

  # get level dict
  lev = []
  level_tolerance = 11 
  for i in range(len(xposlev)):
    if lev == []:
      lev.append({'xPos': [xposlev[i]], 'index': [i]})
    else:
      diff = abs(np.subtract([np.mean(lv['xPos']) for lv in lev], xposlev[i]))
      if np.amin(diff) > level_tolerance:
        lev.append({'xPos': [xposlev[i]], 'index': [i]})
      else:
        idx = np.where(diff == np.amin(diff))[0][0]
        lev[idx]['xPos'].append(xposlev[i])
        lev[idx]['index'].append(i)

  # get level list of int
  level_dict = lev
  srtd_idx = sorted([(lv['index'], np.mean(lv['xPos'])) for lv in level_dict],
          key=lambda tup: tup[1])
  srtd_idx = [idx[0] for idx in srtd_idx]
  levs = []
  t_levs = len(srtd_idx)
  for idx in range(len(xposlev)):
    mask = np.array([idx in tst for tst in srtd_idx])
    n = np.arange(t_levs)[mask]
    levs.append(n[0])

  # for npge, x_, lev, text, labl in zip(npagelev, xposlev, levs, textlev, labellev):
  #   print(npge, x_ if x_ != np.int16(9990) else '---', lev, '>' * lev, [(tx, lb) for tx, lb in zip(text, labl)])
  
  entry = []
  prjopt = {'Project': '', 'Output': ''}

  crdstack = []
  bltstack = []
  for pnum, xpos, level, text, label, idx in zip(npagelev, xposlev, levs, textlev, labellev, range(len(loglev))):
    if 'รายการบุคลากรภาครัฐ' in text:
      print('🚨 line skiped', pnum, level, text, label)
      iserrorlev[idx], loglev[idx] = True, 'LINE SKIPED'
      continue

    if 'วงเงินทั้งสิ้น' in text:
      continue

    if '(1) รายการไม่ผูกพัน' in ' '.join(text):
      continue

    context = ' '.join([txt for lbel, txt in zip(label, text) if lbel == 'NMT'])

    if 'OTP' in label or 'PRJ' in label:
      context = ' '.join([txt for lbel, txt in zip(label, text) if lbel == 'NMT' and txt != ':'])
      if 'OTP' in label:
        prjopt['Output'] = context
      else:
        prjopt['Project'] = context
      continue
    elif 'FAMT' in label or 'FBTH' in label:
      amount = text[label.index('FAMT')] if 'FAMT' in label else '0'
      if entry:
        entry[-1]['is_obliged'] = 1
        if label.count('FYR') == 2:
          year = [txt for lbel, txt in zip(label, text) if lbel == 'FYR']
          fiscal_year = {i:amount for i in range(to_gregorian(year[0]), to_gregorian(year[1])+1)}
        elif label.count('FYR') == 1:
          fiscal_year = {to_gregorian(text[label.index('FYR')]): amount}
        else:
          print('🚨 no \'FYR\'', pnum, level, text, label)
          iserrorlev[idx], loglev[idx] = True, '\'FYR\' NOT FOUND'
          continue

        if isinstance(entry[-1]['fiscal_year'], int):
          entry[-1]['fiscal_year'] = fiscal_year
        elif isinstance(entry[-1]['fiscal_year'], dict):
          entry[-1]['fiscal_year'].update(fiscal_year)
        else:
          print('🚨 type not match', entry[-1]['fiscal_year'])
          iserrorlev[idx], loglev[idx] = True, 'TYPE NOT MATCH'
      else:
        print('🚨 entry is empty', pnum, level, text, label)
        iserrorlev[idx], loglev[idx] = True, 'ENTRY IS EMPTY'
        
      continue

    context = ' '.join([txt for lbel, txt in zip(label, text) if lbel == 'NMT'])
    amount = text[label.index('AMT')] if 'AMT' in label else '0'
    
    temp = {'context': context,
            'ref_line': text,
            'amount': amount,
            'coord_level': level,
            'page': pnum,
            'fiscal_year': int(curr_doc.fiscal_year),
            'is_obliged': 0,
            'DEBUG_LOG': ''}

    blt = 0
    bltlevel = -1
    clevel = level
    bltfound = 0
    if get_level_of_line(''.join(text)):
      bltfound = 1
      if 'BLT' in label:
        blt = text[label.index('BLT')]
      else:
        print('\'BLT\' not found', text )
        iserrorlev[idx], loglev[idx] = True, '\'BLT\' NOT FOUND'
        blt = text[0]
      if blt.count('('):
        clevel = 12
      bltlevel = get_level_of_line(''.join(text))
      temp['bullet_level'] = (bltlevel, blt)
    else:
      temp['DEBUG_LOG'] = 'NO BULLET FOUND'
      iserrorlev[idx], loglev[idx] = True, 'NO BULLET FOUND'

    if clevel == 12:
      temp['crdstack_level'] = 7
    else:
      db = crdstack[:]
      if crdstack:
        if crdstack[-1] < clevel:
          crdstack.append(clevel)
        elif crdstack[-1] > clevel:
          if not clevel in crdstack:
            crdstack[-1] = clevel
          while crdstack[-1] != clevel:
            crdstack.pop()
      else:
        crdstack.append(clevel)
      temp['crdstack_level'] = len(crdstack)

    if bltlevel != -1:
      if bltlevel == 12:
        temp['bltstack_level'] = 7
      else:
        db = bltstack[:]
        if bltstack:
          if bltstack[-1] < bltlevel:
            bltstack.append(bltlevel)
          elif bltstack[-1] > bltlevel:
            if not bltlevel in bltstack:
              bltstack[-1] = bltlevel
            while bltstack[-1] != bltlevel:
              bltstack.pop()
        else:
          bltstack.append(bltlevel)
        temp['bltstack_level'] = len(bltstack)
    else:
      temp['bltstack_level'] = len(bltstack)
      print('🚨 bulltet_lebel is -1: ', text, label)
    if temp['bltstack_level'] != temp['crdstack_level']:
      if bltfound:
        if temp['bltstack_level'] < temp['crdstack_level']:
          temp['level'] = temp['crdstack_level']
        else:
          temp['level'] = temp['bltstack_level']
      else:
        temp['level'] = temp['crdstack_level']

      iserrorlev[idx], loglev[idx] = True, 'LEVEL NOT MATCH'
      log = 'LEVEL NOT MATCH level: by x position {}, by pattern {}'.format(temp['crdstack_level'], temp['bltstack_level'])
      temp['DEBUG_LOG'] = temp['DEBUG_LOG'] + '/ ' + log if temp['DEBUG_LOG'] else log
    else:
      temp['level'] = temp['bltstack_level']

    if temp['DEBUG_LOG']:
      print(temp['DEBUG_LOG'])
      print('page: {}, [\'level\']: {}, xPos: {}, text: {}'.format(pnum, temp['level'], xpos, temp['ref_line']))
    
    entry.append(temp)
  
  semi_raw = []
  for pnum, xpos, level, text, label, iserror, log in zip(npagelev, xposlev, levs, textlev, labellev, iserrorlev, loglev):
    semi_raw.append({'text': text,
          'minxpos': xpos,
          'label': label,
          'page_number': pnum,
          'error_found': iserror,
          'log': log})

  return prjopt, entry, semi_raw

In [42]:
 %%time
tbg = ThaiBudget('../../txt-extraction/tee4cute-gcloud-vision/output/budget_pdf2txt_with_coordinates.json')

CPU times: user 19.2 s, sys: 4.26 s, total: 23.5 s
Wall time: 24.4 s


In [48]:
curr_doc = tbg.get_doc('2022.3.3(1)')
lemtestrange = curr_doc.range
for i, lrange in enumerate(lemtestrange[:]):
    print(i, lrange['budget_unit'], lrange['range'])

pstart = curr_doc.start
listpage = []
for lrange in lemtestrange:
    startp, endp = lrange['range']
    for ipage in range(startp, endp):
        listpage.append(curr_doc.page(ipage + pstart))
        if ipage == 619:
          listpage[-1].dline_tolerance = 29
        listpage[-1].pdfpage = ipage

0 สำนักงานปลัดกระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกรรม (40, 173)
1 กรมวิทยาศาสตร์บริการ (183, 225)
2 สำนักงานการวิจัยแห่งชาติ (232, 242)
3 สำนักงานปรมาณูเพื่อสันติ (249, 268)
4 สถาบันวิทยาลัยชุมชน (278, 321)
5 มหาวิทยาลัยกาฬสินธุ์ (330, 342)
6 มหาวิทยาลัยนครพนม (348, 361)
7 มหาวิทยาลัยนราธิวาสราชนครินทร์ (370, 391)
8 มหาวิทยาลัยนเรศวร (399, 422)
9 มหาวิทยาลัยมหาสารคาม (429, 444)
10 มหาวิทยาลัยรามคำแหง (450, 458)
11 มหาวิทยาลัยสุโขทัยธรรมาธิราช (464, 470)
12 มหาวิทยาลัยอุบลราชธานี (479, 506)
13 สถาบันเทคโนโลยีปทุมวัน (511, 518)
14 มหาวิทยาลัยราชภัฏกาญจนบุรี (524, 540)
15 มหาวิทยาลัยราชภัฏกำแพงเพชร (548, 565)
16 มหาวิทยาลัยราชภัฏจันทรเกษม (573, 593)
17 มหาวิทยาลัยราชภัฏชัยภูมิ (600, 621)
18 มหาวิทยาลัยราชภัฏเชียงราย (630, 649)
19 มหาวิทยาลัยราชภัฏเชียงใหม่ (658, 680)


In [49]:
idtpages = []
ihdpages = []
tmphdict = {}
semi_raw_entry = {curr_doc.ref_doc :[]}
Header = []
Detail = []

bgplan = ''
lemrange = lemtestrange[:]

for ipage in tqdm(listpage):
    text = '\n'.join(ipage.get_text_lines())
    headerw = ['หน่วย : ล้านบาท', 'หน่วยเล้านบาท', 'วัตถุประสงค์', 'หน่วย:ล้านบาท', 'หน่วยนับ']
    if not [hw for hw in headerw if hw in text]:
        # detail
        if ihdpages:
          tbgplan = get_budget_plan(ihdpages)
          bgplan = tbgplan if tbgplan else bgplan

          budget_unit = ''
          ministry = ''
          while not budget_unit:
            if lemrange[0]['range'][0] <= ihdpages[0].pdfpage < lemrange[0]['range'][1]:
              ministry = lemrange[0]['ministry']
              budget_unit = lemrange[0]['budget_unit']
            else:
              lemrange.pop(0)
          
          tmphdict = {'ministry': ministry, 'budget_unit': budget_unit,
                      'budget_plan': bgplan, 'range': [i.pdfpage for i in ihdpages],
                      'is_cross_func': bgplan.startswith('แผนงานบูรณาการ')}
          ihdpages = []
        idtpages.append(ipage)
    else:
        # header
        if idtpages:
          prjopt, entry, semi_raw = get_entry(clean_entry(idtpages))
          semi_raw_entry[curr_doc.ref_doc].append(semi_raw)
          Detail.append(entry)
          tmphdict.update(prjopt)
          Header.append(tmphdict)
          idtpages = []
        ihdpages.append(ipage)
if idtpages:
  prjopt, entry, semi_raw = get_entry(clean_entry(idtpages))
  semi_raw_entry[curr_doc.ref_doc].append(semi_raw)
  Detail.append(entry)
  tmphdict.update(prjopt)
  Header.append(tmphdict)

LEVEL NOT MATCH level: by x position 4, by pattern 2
page: 50, ['level']: 4, xPos: 272, text: ['2.1.', 'ค่าควบคุมงาน', '1,350,000', 'บาท']
LEVEL NOT MATCH level: by x position 4, by pattern 3
page: 50, ['level']: 4, xPos: 271, text: ['2.1.2.2', 'ค่าปรับปรุงสิ่งก่อสร้างอื่น', '13,365,000', 'บาท']
LEVEL NOT MATCH level: by x position 4, by pattern 3
page: 50, ['level']: 4, xPos: 271, text: ['2.1.2.3', 'ค่าปรับปรุงอาคารที่ทำการและสิ่งก่อสร้างประกอบ', '30,000,000', 'บาท']
🚨 trash line ['บ', 'น']
🚨 trash line ['บ', 'ข']
🚨 trash line ['6', 'น']
🚨 trash line ['ปี', '2563']
🚨 no 'FYR' 109 5 ['เงินงบประมาณ', '75,846,000', 'Uın', 'ตั้งงบประมาณ', 'บ', 'ะ', '13,518,000', 'บาท'] ['NMT', 'NMT', 'NMT', 'NMT', 'NMT', 'NMT', 'FAMT', 'FBTH']
🚨 trash line ['シ', 'シ']
🚨 trash line ['121', 'มี']
🚨 trash line ['6', 'บ']
🚨 trash line ['บ', 'ข']
🚨 trash line ['ปี']
🚨 trash line ['บ']
🚨 trash line ['6ส่d']
🚨 trash line ['ปี']
🚨 trash line ['ปี']
🚨 trash line ['บส่']
🚨 trash line ['ข', '0']
🚨 trash line ['de']
🚨

In [50]:
print('ENTRIES CONTAIN ERROR')
for bu in semi_raw_entry[curr_doc.ref_doc]:
  for i in bu:
    if i['error_found']:
      print(i)

ENTRIES CONTAIN ERROR
{'text': ['2.1.', 'ค่าควบคุมงาน', '1,350,000', 'บาท'], 'minxpos': 272, 'label': ['BLT', 'NMT', 'AMT', 'BTH'], 'page_number': 50, 'error_found': True, 'log': 'LEVEL NOT MATCH'}
{'text': ['2.1.2.2', 'ค่าปรับปรุงสิ่งก่อสร้างอื่น', '13,365,000', 'บาท'], 'minxpos': 271, 'label': ['BLT', 'NMT', 'AMT', 'BTH'], 'page_number': 50, 'error_found': True, 'log': 'LEVEL NOT MATCH'}
{'text': ['2.1.2.3', 'ค่าปรับปรุงอาคารที่ทำการและสิ่งก่อสร้างประกอบ', '30,000,000', 'บาท'], 'minxpos': 271, 'label': ['BLT', 'NMT', 'AMT', 'BTH'], 'page_number': 50, 'error_found': True, 'log': 'LEVEL NOT MATCH'}
{'text': ['เงินงบประมาณ', '75,846,000', 'Uın', 'ตั้งงบประมาณ', 'บ', 'ะ', '13,518,000', 'บาท'], 'minxpos': 319, 'label': ['NMT', 'NMT', 'NMT', 'NMT', 'NMT', 'NMT', 'FAMT', 'FBTH'], 'page_number': 109, 'error_found': True, 'log': "'FYR' NOT FOUND"}
{'text': ['เครื่องวัดความหนาแน่นของกระดูก', 'ตำบลตลาด', 'อำเภอเมืองมหาสารคาม', 'จังหวัดมหาสารคาม', '1', 'เครื่อง', '4,500,000', 'บาท'], 'minxpos': 

In [51]:
def get_categorys_lv(list_of_dict):
  CATEGORY_LV1, CATEGORY_LV2, CATEGORY_LV3, CATEGORY_LV4, CATEGORY_LV5, CATEGORY_LV6, ITEM_DESCRIPTION, DEBUG_LOG = [''] * 8
  for ite in list_of_dict:
    if ite['DEBUG_LOG']:
      DEBUG_LOG += ite['DEBUG_LOG']
    if ite['level'] == 1:
      CATEGORY_LV1 = ite['context']
    elif ite['level'] == 2:
      CATEGORY_LV2 = ite['context']
    elif ite['level'] == 3:
      CATEGORY_LV3 = ite['context']
    elif ite['level'] == 4:
      CATEGORY_LV4 = ite['context']
    elif ite['level'] == 5:
      CATEGORY_LV5 = ite['context']
    elif ite['level'] == 6:
      CATEGORY_LV6 = ite['context']
    elif ite['level'] == 7:
      ITEM_DESCRIPTION = ite['context']
  if DEBUG_LOG:
    DEBUG_LOG += '/ {}'.format([ite['ref_line'] for ite in list_of_dict])
  return CATEGORY_LV1, CATEGORY_LV2, CATEGORY_LV3, CATEGORY_LV4, CATEGORY_LV5, CATEGORY_LV6, ITEM_DESCRIPTION, DEBUG_LOG

In [52]:
def get_rows_of_budg_unit(running_id, budget_plan_details, bud_u):

  REF_DOC = curr_doc.ref_doc
  ITEM_ID = '{}.{}'.format(REF_DOC, running_id)
  REF_PAGE_NO = -1
  DEBUG_LOG = ''
  MINISTRY = budget_plan_details['ministry']
  BUDGETARY_UNIT = budget_plan_details['budget_unit']
  CROSS_FUNC = budget_plan_details['is_cross_func']
  BUDGET_PLAN = budget_plan_details['budget_plan']
  OUTPUT = budget_plan_details['Output']
  PROJECT = budget_plan_details['Project']

  CATEGORY_LV1, CATEGORY_LV2, CATEGORY_LV3, CATEGORY_LV4, CATEGORY_LV5, CATEGORY_LV6, ITEM_DESCRIPTION = [''] * 7
  FISCAL_YEAR = curr_doc.fiscal_year
  AMOUNT = -1
  OBLIGED = 0

  stack = []
  rows = []
  for item in bud_u:
    if stack:
      if stack[-1]['level'] < item['level']:
        stack.append(item)
      elif stack[-1]['level'] > item['level']:
        rows.append([x for x in stack])
        stack.pop()
      elif stack[-1]['level'] == item['level']:
        rows.append([x for x in stack])
      if stack:
        stack.pop()
      stack.append(item)
    else:
      stack.append(item)
      
  rows.append([x for x in stack])

  rows_of_lists = []

  for row in rows[:]:
    if not row:
      continue
    itms = row[-1]
    REF_PAGE_NO = np.int16(itms['page'])
    OBLIGED = itms['is_obliged']
    CATEGORY_LV1, CATEGORY_LV2, CATEGORY_LV3, CATEGORY_LV4, CATEGORY_LV5, CATEGORY_LV6, ITEM_DESCRIPTION, DEBUG_LOG = get_categorys_lv([x for x in row])

    if itms['is_obliged']:
      for fy, amt in itms['fiscal_year'].items():
        FISCAL_YEAR = fy
        AMOUNT = amt
        ITEM_ID = '{}.{}'.format(REF_DOC, running_id)
        running_id += 1
        rows_of_lists.append([ITEM_ID, REF_DOC, np.int16(REF_PAGE_NO), MINISTRY, BUDGETARY_UNIT, np.bool_(CROSS_FUNC), BUDGET_PLAN, OUTPUT,
                              PROJECT, CATEGORY_LV1, CATEGORY_LV2, CATEGORY_LV3, CATEGORY_LV4,
                              CATEGORY_LV5, CATEGORY_LV6, ITEM_DESCRIPTION,
                              FISCAL_YEAR, AMOUNT, np.bool_(OBLIGED), DEBUG_LOG])
    else:
      FISCAL_YEAR = itms['fiscal_year']
      AMOUNT = itms['amount']
      ITEM_ID = '{}.{}'.format(REF_DOC, running_id)
      running_id += 1
      rows_of_lists.append([ITEM_ID, REF_DOC, np.int16(REF_PAGE_NO), MINISTRY, BUDGETARY_UNIT, np.bool_(CROSS_FUNC), BUDGET_PLAN, OUTPUT,
                            PROJECT, CATEGORY_LV1, CATEGORY_LV2, CATEGORY_LV3, CATEGORY_LV4,
                            CATEGORY_LV5, CATEGORY_LV6, ITEM_DESCRIPTION, FISCAL_YEAR, AMOUNT, np.bool_(OBLIGED), DEBUG_LOG])
  return running_id, rows_of_lists

In [53]:
cols = ['ITEM_ID','REF_DOC','REF_PAGE_NO','MINISTRY','BUDGETARY_UNIT',
        'CROSS_FUNC?','BUDGET_PLAN','OUTPUT','PROJECT','CATEGORY_LV1',
        'CATEGORY_LV2','CATEGORY_LV3','CATEGORY_LV4','CATEGORY_LV5',
        'CATEGORY_LV6','ITEM_DESCRIPTION','FISCAL_YEAR','AMOUNT','OBLIGED?', 'DEBUG_LOG']

df = pd.DataFrame(columns=cols)

running_id = 0
for header, detail in zip(Header, Detail):
  running_id, rows = get_rows_of_budg_unit(running_id, header, detail)
  new_row = pd.DataFrame(rows, columns=cols)
  df = df.append(new_row, ignore_index=True)

In [54]:
df.loc[df['DEBUG_LOG'] == ''].sample(5)

,ITEM_ID,REF_DOC,REF_PAGE_NO,MINISTRY,BUDGETARY_UNIT,CROSS_FUNC?,BUDGET_PLAN,OUTPUT,PROJECT,CATEGORY_LV1,CATEGORY_LV2,CATEGORY_LV3,CATEGORY_LV4,CATEGORY_LV5,CATEGORY_LV6,ITEM_DESCRIPTION,FISCAL_YEAR,AMOUNT,OBLIGED?,DEBUG_LOG
1264,2022.3.3(1).1264,2022.3.3(1),670,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,มหาวิทยาลัยราชภัฏเชียงใหม่,False,แผนงานยุทธศาสตร์เสริมสร้างพลังทางสังคม,,โครงการยุทธศาสตร์มหาวิทยาลัยราชภัฏเพื่อการพัฒน...,งบเงินอุดหนุน,เงินอุดหนุนทั่วไป,เงินอุดหนุนค่าใช้จ่ายโครงการยุทธศาสตร์มหาวิทยา...,,,,,2022,00,False,
882,2022.3.3(1).882,2022.3.3(1),465,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,มหาวิทยาลัยสุโขทัยธรรมาธิราช,False,แผนงานบุคลากรภาครัฐ,,,งบบุคลากร,เงินเดือนและค่าจ้างประจำ,เงินเดือน,,,,,2022,"241,888,000",False,
183,2022.3.3(1).183,2022.3.3(1),115,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,สำนักงานปลัดกระทรวงการอุดมศึกษา วิทยาศาสตร์ วิ...,False,แผนงานยุทธศาสตร์เพื่อสนับสนุนด้านการสร้างความส...,,โครงการเสริมสร้างศักยภาพของบุคลากร ด้านวิทยาศา...,งบเงินอุดหนุน,เงินอุดหนุนทั่วไป,เงินอุดหนุนเป็นค่าใช้จ่ายเพื่อบริหารสำนักวิจัย...,,,,,2022,"26,500,000",False,
489,2022.3.3(1).489,2022.3.3(1),258,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,สำนักงานปรมาณูเพื่อสันติ,False,แผนงานยุทธศาสตร์เพื่อสนับสนุนด้านการสร้างความส...,การสร้างมาตรการความปลอดภัยในการกำกับดูแล ความป...,,งบลงทุน,ค่าครุภัณฑ์ ที่ดินและสิ่งก่อสร้าง,ค่าครุภัณฑ์,ครุภัณฑ์สำนักงาน,,,ลิฟต์โดยสาร ประจำอาคาร 4 และอาคาร 9 แขวงลาดยาว...,2022,"3,440,000",False,
12,2022.3.3(1).12,2022.3.3(1),44,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,สำนักงานปลัดกระทรวงการอุดมศึกษา วิทยาศาสตร์ วิ...,False,แผนงานพื้นฐานด้านการสร้างความสามารถในการแข่งขัน,การถ่ายทอดเทคโนโลยี,,งบดำเนินงาน,ค่าตอบแทน ใช้สอยและวัสดุ,,,,,ค่าใช้จ่ายในการนำวิทยาศาสตร์ เทคโนโลยี และนวัต...,2022,"5,000,000",False,


In [55]:
df.loc[df['DEBUG_LOG'] != '']

,ITEM_ID,REF_DOC,REF_PAGE_NO,MINISTRY,BUDGETARY_UNIT,CROSS_FUNC?,BUDGET_PLAN,OUTPUT,PROJECT,CATEGORY_LV1,CATEGORY_LV2,CATEGORY_LV3,CATEGORY_LV4,CATEGORY_LV5,CATEGORY_LV6,ITEM_DESCRIPTION,FISCAL_YEAR,AMOUNT,OBLIGED?,DEBUG_LOG
58,2022.3.3(1).58,2022.3.3(1),50,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,สำนักงานปลัดกระทรวงการอุดมศึกษา วิทยาศาสตร์ วิ...,False,แผนงานพื้นฐานด้านการสร้างความสามารถในการแข่งขัน,ข้อเสนอแนะด้านนโยบายและแผน,,งบลงทุน,ค่าครุภัณฑ์ ที่ดินและสิ่งก่อสร้าง,ค่าที่ดินและสิ่งก่อสร้าง ส่ล,ค่าควบคุมงาน,,,ค่าควบคุมงานที่มีราคาต่อหน่วยต่ำกว่า 10 ล้านบา...,2022,"1,350,000",False,"LEVEL NOT MATCH level: by x position 4, by pat..."
59,2022.3.3(1).59,2022.3.3(1),50,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,สำนักงานปลัดกระทรวงการอุดมศึกษา วิทยาศาสตร์ วิ...,False,แผนงานพื้นฐานด้านการสร้างความสามารถในการแข่งขัน,ข้อเสนอแนะด้านนโยบายและแผน,,งบลงทุน,ค่าครุภัณฑ์ ที่ดินและสิ่งก่อสร้าง,ค่าที่ดินและสิ่งก่อสร้าง ส่ล,ค่าปรับปรุงสิ่งก่อสร้างอื่น,,,ค่าปรับปรุงสิ่งก่อสร้างอื่นที่มีราคาต่อหน่วยต่...,2022,"13,365,000",False,"LEVEL NOT MATCH level: by x position 4, by pat..."
60,2022.3.3(1).60,2022.3.3(1),50,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,สำนักงานปลัดกระทรวงการอุดมศึกษา วิทยาศาสตร์ วิ...,False,แผนงานพื้นฐานด้านการสร้างความสามารถในการแข่งขัน,ข้อเสนอแนะด้านนโยบายและแผน,,งบลงทุน,ค่าครุภัณฑ์ ที่ดินและสิ่งก่อสร้าง,ค่าที่ดินและสิ่งก่อสร้าง ส่ล,ค่าปรับปรุงอาคารที่ทำการและสิ่งก่อสร้างประกอบ,,,ค่าปรับปรุงอาคารพระจอมเกล้าชัน 6 แขวงทุ่งพญาไท...,2022,"15,000,000",False,"LEVEL NOT MATCH level: by x position 4, by pat..."
61,2022.3.3(1).61,2022.3.3(1),50,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,สำนักงานปลัดกระทรวงการอุดมศึกษา วิทยาศาสตร์ วิ...,False,แผนงานพื้นฐานด้านการสร้างความสามารถในการแข่งขัน,ข้อเสนอแนะด้านนโยบายและแผน,,งบลงทุน,ค่าครุภัณฑ์ ที่ดินและสิ่งก่อสร้าง,ค่าที่ดินและสิ่งก่อสร้าง ส่ล,ค่าปรับปรุงอาคารที่ทำการและสิ่งก่อสร้างประกอบ,,,ค่าปรับปรุงอาคารพระจอมเกล้าชั้น 7 แขวงทุ่งพญาไ...,2022,"15,000,000",False,"LEVEL NOT MATCH level: by x position 4, by pat..."
822,2022.3.3(1).822,2022.3.3(1),440,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,มหาวิทยาลัยมหาสารคาม,False,แผนงานยุทธศาสตร์เสริมสร้างให้คนมีสุขภาวะที่ดี,,โครงการเพิ่มศักยภาพการให้บริการทางด้านสาธารณสุข,งบลงทุน,ค่าครุภัณฑ์ ที่ดินและสิ่งก่อสร้าง,ค่าครุภัณฑ์,,เครื่องวัดความหนาแน่นของกระดูก ตำบลตลาด อำเภอเ...,,เครื่องฉายแสงอุลตร้าไวโอเลต เฉพาะบริเวณเพื่อรั...,2022,"1,900,000",False,NO BULLET FOUND/ LEVEL NOT MATCH level: by x p...
823,2022.3.3(1).823,2022.3.3(1),440,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,มหาวิทยาลัยมหาสารคาม,False,แผนงานยุทธศาสตร์เสริมสร้างให้คนมีสุขภาวะที่ดี,,โครงการเพิ่มศักยภาพการให้บริการทางด้านสาธารณสุข,งบลงทุน,ค่าครุภัณฑ์ ที่ดินและสิ่งก่อสร้าง,ค่าครุภัณฑ์,,เครื่องวัดความหนาแน่นของกระดูก ตำบลตลาด อำเภอเ...,,เครื่องตรวจหาชนิดของเชื้อแบบอัตโนมัติ ตำบลตลาด...,2022,"7,490,000",False,NO BULLET FOUND/ LEVEL NOT MATCH level: by x p...
824,2022.3.3(1).824,2022.3.3(1),440,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,มหาวิทยาลัยมหาสารคาม,False,แผนงานยุทธศาสตร์เสริมสร้างให้คนมีสุขภาวะที่ดี,,โครงการเพิ่มศักยภาพการให้บริการทางด้านสาธารณสุข,งบลงทุน,ค่าครุภัณฑ์ ที่ดินและสิ่งก่อสร้าง,ค่าครุภัณฑ์,,เครื่องวัดความหนาแน่นของกระดูก ตำบลตลาด อำเภอเ...,,เครื่องปั่นแยกส่วนประกอบของเลือด ขนาดความจุไม่...,2022,"2,100,000",False,NO BULLET FOUND/ LEVEL NOT MATCH level: by x p...
825,2022.3.3(1).825,2022.3.3(1),440,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,มหาวิทยาลัยมหาสารคาม,False,แผนงานยุทธศาสตร์เสริมสร้างให้คนมีสุขภาวะที่ดี,,โครงการเพิ่มศักยภาพการให้บริการทางด้านสาธารณสุข,งบลงทุน,ค่าครุภัณฑ์ ที่ดินและสิ่งก่อสร้าง,ค่าครุภัณฑ์,,เครื่องมือถ่างเนื้อเยื่อสำหรับคอส่วนหน้า ตำบลต...,,เครื่องมือสำหรับผ่าตัดแบบแผลเล็ก ตำบลตลาด อำเภ...,2022,"2,242,000",False,NO BULLET FOUND/ LEVEL NOT MATCH level: by x p...
826,2022.3.3(1).826,2022.3.3(1),440,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,มหาวิทยาลัยมหาสารคาม,False,แผนงานยุทธศาสตร์เสริมสร้างให้คนมีสุขภาวะที่ดี,,โครงการเพิ่มศักยภาพการให้บริการทางด้านสาธารณสุข,งบลงทุน,ค่าครุภัณฑ์ ที่ดินและสิ

In [56]:
df.to_csv('{}.csv'.format(curr_doc.ref_doc.replace('.', '-')))

In [ ]:
xposl, textl = curr_doc.page(9).xpos_text_lines()
for xpos, line in zip(xposl, textl):
  print([(txt, x) for x, txt in zip(xpos.tolist(), line)])

In [57]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

In [62]:
skip = True
if not skip:
  OJSON = []
  for header, detail in zip (Header, Detail):
    OJSON.append(header)
    OJSON[-1].update({'entry': detail})

  with open("{}.json".format(curr_doc.ref_doc.replace('.', '-')), "w", encoding="utf8") as file:
    json.dump(OJSON, file, ensure_ascii=False, cls=NpEncoder)

In [63]:
with open("{}-raw-entry.json".format(curr_doc.ref_doc.replace('.', '-')), "w", encoding="utf8") as file:
  json.dump(semi_raw_entry, file, ensure_ascii=False, cls=NpEncoder)